In [ ]:
# Check GPU
!nvidia-smi

# Fresh install Ultralytics with YOLOv11
!pip -q install --upgrade ultralytics

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import torch, os, sys, yaml, glob, random, shutil, math, json, textwrap
print("Torch:", torch.__version__)
import ultralytics
from ultralytics import YOLO
print("Ultralytics:", ultralytics.__version__)

# Ensure CUDA available
assert torch.cuda.is_available(), "GPU not detected. In Colab: Runtime > Change runtime type > Hardware accelerator: GPU"


Thu Oct 30 13:02:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 🔧 EDIT THESE to your actual dataset location in Drive
# Example:
# /content/drive/MyDrive/datasets/soil-detection
DATASET_DIR = "/content/drive/MyDrive/SoilDetectionDataset"   # <- change me
DATA_YAML    = os.path.join(DATASET_DIR, "data.yaml")           # <- change me if yaml is elsewhere

# Where to save runs
PROJECT_DIR = "/content/drive/MyDrive/yolo11_soil_runs"
os.makedirs(PROJECT_DIR, exist_ok=True)

# Sanity prints
print("DATASET_DIR:", DATASET_DIR)
print("DATA_YAML:", DATA_YAML)
print("PROJECT_DIR:", PROJECT_DIR)


DATASET_DIR: /content/drive/MyDrive/SoilDetectionDataset
DATA_YAML: /content/drive/MyDrive/SoilDetectionDataset/data.yaml
PROJECT_DIR: /content/drive/MyDrive/yolo11_soil_runs


In [ ]:
# Read YAML (classes, paths)
assert os.path.exists(DATA_YAML), f"data.yaml not found at: {DATA_YAML}"

with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)

print("data.yaml content:\n", json.dumps(data_cfg, indent=2))

# Basic checks
for k in ["train", "val", "names"]:
    assert k in data_cfg, f"Missing '{k}' key in data.yaml"

def count_images_labels(img_dir):
    images = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.JPG","*.PNG"):
        images += glob.glob(os.path.join(img_dir, ext))
    labels_dir = img_dir.replace("/images", "/labels")
    label_files = glob.glob(os.path.join(labels_dir, "*.txt"))
    return len(images), len(label_files)

def report_split(split_key):
    path = data_cfg[split_key]
    # If YAML uses relative dirs, resolve relative to DATASET_DIR
    if not os.path.isabs(path):
        path = os.path.join(DATASET_DIR, path)
    # If it's a text file listing images, count lines; else assume folder
    if os.path.isfile(path) and path.endswith(".txt"):
        with open(path) as f:
            imgs = [l.strip() for l in f if l.strip()]
        print(f"{split_key}: file list -> {len(imgs)} images")
    else:
        img_dir = os.path.join(path, "images") if os.path.isdir(path) else path
        n_img, n_lab = count_images_labels(img_dir)
        print(f"{split_key}: images={n_img}, labels={n_lab}, path={path}")

report_split("train")
report_split("val")
if "test" in data_cfg:
    report_split("test")

print("Classes:", data_cfg["names"])


data.yaml content:
 {
  "root": "/content/drive/MyDrive/SoilDetectionDataset",
  "train": "/train/images",
  "val": "/valid/images",
  "test": "/test/images",
  "nc": 4,
  "names": [
    "Alluvial Soil",
    "Black Soil",
    "Clay Soil",
    "Red Soil"
  ],
  "roboflow": {
    "workspace": "imit-h5ows",
    "project": "soil-detection-l0ngo",
    "version": 3,
    "license": "CC BY 4.0",
    "url": "https://universe.roboflow.com/imit-h5ows/soil-detection-l0ngo/dataset/3"
  }
}
train: images=0, labels=0, path=/train/images
val: images=0, labels=0, path=/valid/images
test: images=0, labels=0, path=/test/images
Classes: ['Alluvial Soil', 'Black Soil', 'Clay Soil', 'Red Soil']


1) Fix & normalize data.yaml paths using root

In [ ]:
import os, yaml, json, glob

assert os.path.exists(DATA_YAML), f"data.yaml not found at: {DATA_YAML}"

with open(DATA_YAML, "r") as f:
    raw = yaml.safe_load(f)

print("Original YAML:\n", json.dumps(raw, indent=2))

ROOT = raw.get("root", "")
assert ROOT and os.path.exists(ROOT), f"'root' missing or not found: {ROOT}"

def norm_path(p):
    if p is None:
        return None
    # If it starts with "/", it's trying to be absolute (e.g. "/train/images").
    # Interpret it as relative to ROOT.
    if isinstance(p, str) and p.startswith("/"):
        p = p[1:]  # drop leading slash
        return os.path.join(ROOT, p)
    # If it's already absolute and exists, keep it; else resolve relative to ROOT.
    if os.path.isabs(p) and os.path.exists(p):
        return p
    return os.path.join(ROOT, p)

fixed = dict(raw)  # copy
for key in ["train", "val", "test"]:
    if key in fixed and fixed[key]:
        fixed[key] = norm_path(fixed[key])

# Optional: ensure Ultralytics can read this even if paths point to ".../images"
# (Ultralytics accepts images-folder paths; labels are inferred from sibling ".../labels")
fixed["nc"] = fixed.get("nc", len(fixed.get("names", [])))

ABS_YAML = "/content/data_abs.yaml"
with open(ABS_YAML, "w") as f:
    yaml.safe_dump(
        {k: fixed[k] for k in ["train","val","test","names","nc"] if k in fixed},
        f, sort_keys=False
    )

print("\n✅ Wrote normalized YAML to:", ABS_YAML)
print("Resolved paths:\n", json.dumps({k: fixed.get(k) for k in ["train","val","test"]}, indent=2))
print("Classes:", fixed.get("names"))


Original YAML:
 {
  "root": "/content/drive/MyDrive/SoilDetectionDataset",
  "train": "/train/images",
  "val": "/valid/images",
  "test": "/test/images",
  "nc": 4,
  "names": [
    "Alluvial Soil",
    "Black Soil",
    "Clay Soil",
    "Red Soil"
  ],
  "roboflow": {
    "workspace": "imit-h5ows",
    "project": "soil-detection-l0ngo",
    "version": 3,
    "license": "CC BY 4.0",
    "url": "https://universe.roboflow.com/imit-h5ows/soil-detection-l0ngo/dataset/3"
  }
}

✅ Wrote normalized YAML to: /content/data_abs.yaml
Resolved paths:
 {
  "train": "/content/drive/MyDrive/SoilDetectionDataset/train/images",
  "val": "/content/drive/MyDrive/SoilDetectionDataset/valid/images",
  "test": "/content/drive/MyDrive/SoilDetectionDataset/test/images"
}
Classes: ['Alluvial Soil', 'Black Soil', 'Clay Soil', 'Red Soil']


In [ ]:
import os, glob

def count_split(path_or_file):
    if path_or_file is None:
        return 0,0,"",""
    # If it's a file list:
    if os.path.isfile(path_or_file) and path_or_file.endswith(".txt"):
        with open(path_or_file) as f:
            imgs = [l.strip() for l in f if l.strip()]
        return len(imgs), None, path_or_file, "(filelist)"
    # If it's a directory:
    img_dir = path_or_file
    if os.path.isdir(os.path.join(path_or_file, "images")):
        img_dir = os.path.join(path_or_file, "images")
    label_dir = img_dir.replace("/images","/labels").replace("\\images","\\labels")

    imgs = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp","*.JPG","*.PNG"):
        imgs += glob.glob(os.path.join(img_dir, ext))
    labs = glob.glob(os.path.join(label_dir, "*.txt"))
    return len(imgs), len(labs), img_dir, label_dir

for split in ["train","val","test"]:
    if split in fixed and fixed[split]:
        n_img, n_lab, i_dir, l_dir = count_split(fixed[split])
        print(f"{split}: images={n_img}, labels={n_lab} | img_dir={i_dir} | lbl_dir={l_dir}")


train: images=199, labels=199 | img_dir=/content/drive/MyDrive/SoilDetectionDataset/train/images | lbl_dir=/content/drive/MyDrive/SoilDetectionDataset/train/labels
val: images=58, labels=58 | img_dir=/content/drive/MyDrive/SoilDetectionDataset/valid/images | lbl_dir=/content/drive/MyDrive/SoilDetectionDataset/valid/labels
test: images=28, labels=28 | img_dir=/content/drive/MyDrive/SoilDetectionDataset/test/images | lbl_dir=/content/drive/MyDrive/SoilDetectionDataset/test/labels


In [ ]:
import os, glob, random, cv2
from pathlib import Path

def preview_samples(split_path, names, n=8, out_dir="/content/_previews", max_side=900):
    os.makedirs(out_dir, exist_ok=True)

    # build list
    if os.path.isfile(split_path) and split_path.endswith(".txt"):
        with open(split_path) as f:
            img_paths = [l.strip() for l in f if l.strip()]
    else:
        img_dir = split_path
        if os.path.isdir(os.path.join(split_path, "images")):
            img_dir = os.path.join(split_path, "images")
        img_paths = []
        for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp","*.JPG","*.PNG"):
            img_paths += glob.glob(os.path.join(img_dir, ext))

    random.shuffle(img_paths)
    img_paths = img_paths[:n]

    def draw(img, xyxy, cls_id):
        x1,y1,x2,y2 = map(int, xyxy)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        label = names[cls_id] if 0 <= cls_id < len(names) else str(cls_id)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(img, (x1, y1-th-6), (x1+tw+6, y1), (0,255,0), -1)
        cv2.putText(img, label, (x1+3, y1-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

    saved = 0
    for p in img_paths:
        img = cv2.imread(p)
        if img is None:
            continue
        h, w = img.shape[:2]
        lbl = p.replace("/images/","/labels/").replace("\\images\\","\\labels\\")
        lbl = os.path.splitext(lbl)[0] + ".txt"

        if os.path.exists(lbl):
            with open(lbl) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    c, cx, cy, bw, bh = map(float, parts)
                    c = int(c)
                    x1 = (cx - bw/2) * w
                    y1 = (cy - bh/2) * h
                    x2 = (cx + bw/2) * w
                    y2 = (cy + bh/2) * h
                    draw(img, (x1,y1,x2,y2), c)

        scale = min(max_side / max(h,w), 1.0)
        if scale < 1.0:
            img = cv2.resize(img, (int(w*scale), int(h*scale)))
        out_path = os.path.join(out_dir, Path(p).name)
        cv2.imwrite(out_path, img)
        saved += 1
    print(f"✅ Saved {saved} previews to {out_dir}")

# Preview TRAIN split
preview_samples(fixed["train"], fixed["names"], n=8)


✅ Saved 8 previews to /content/_previews


In [ ]:
# 1) Install/Import
!pip -q install --upgrade ultralytics
from ultralytics import YOLO
import os, yaml, json, math, random

PROJECT_DIR = "/content/drive/MyDrive/yolo11_soil_runs"
os.makedirs(PROJECT_DIR, exist_ok=True)

# 2) Create a tuned hyperparameter file
# Notes:
# - Higher imgsz helps small textures/patterns (soil)
# - Strong augments early (mosaic/mixup), then close mosaic at the end
# - Slight label smoothing improves generalization
# - AdamW often works well for fine-grained textures; try SGD if needed

hyp = {
  "lr0": 0.0025,            # initial learning rate (a bit lower = stabler)
  "lrf": 0.15,              # final OneCycle LR fraction
  "momentum": 0.937,
  "weight_decay": 0.0005,   # AdamW decoupled weight decay
  "warmup_epochs": 3.0,
  "warmup_momentum": 0.8,
  "warmup_bias_lr": 0.05,

  # Loss balance (slightly upweight objectness/cls for fine textures)
  "box": 7.5,
  "cls": 1.2,
  "dfl": 1.5,

  # Augmentations (strong early, closed late via close_mosaic)
  "hsv_h": 0.015,
  "hsv_s": 0.7,
  "hsv_v": 0.4,
  "degrees": 0.0,
  "translate": 0.08,
  "scale": 0.6,
  "shear": 0.0,
  "perspective": 0.000,
  "flipud": 0.0,
  "fliplr": 0.5,
  "mosaic": 1.0,
  "mixup": 0.10,            # helpful when classes look similar
  "copy_paste": 0.0,        # (mainly for seg; leave 0 for det)

  # Regularization
  "label_smoothing": 0.01,  # a touch of smoothing helps
}

HYP_YAML = "/content/hyp_soil.yaml"
with open(HYP_YAML, "w") as f:
    yaml.safe_dump(hyp, f, sort_keys=False)
print("Saved HYP to:", HYP_YAML)


Saved HYP to: /content/hyp_soil.yaml


In [ ]:
# 3) Choose a stronger base model + high resolution
# Start with 'm' (medium). If VRAM OK, try 'l'.
BASE_MODEL = "yolo11m.pt"   # try yolo11l.pt if you have more GPU memory
IMGSZ      = 896            # 896/960/1024 — pick what your GPU can handle
EPOCHS     = 150            # longer training for higher mAP
BATCH      = -1             # auto-batch to fit VRAM

model = YOLO(BASE_MODEL)

In [ ]:
!pip -q install --upgrade ultralytics
from ultralytics import YOLO
import os



results = model.train(
    data=ABS_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0,
    project=PROJECT_DIR,
    name="yolo11_soil_det_hq",
    seed=42,
    pretrained=True,
    optimizer="AdamW",
    cos_lr=True,
    amp=True,
    cache=True,
    workers=8,
    patience=50,
    plots=True,
    val=True,
    close_mosaic=15,     # turn off mosaic near the end

    # 🔧 Hyperparameters (inline instead of hyp=...)
    lr0=0.0025,
    lrf=0.15,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.05,

    # 📦 Loss balance
    box=7.5,
    cls=1.2,
    dfl=1.5,

    # 🎨 Augmentations
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.08,
    scale=0.6,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.10,

    # 🧽 Regularization
    label_smoothing=0.01,
)

print("Training done. Best:", results)


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=15, cls=1.2, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/data_abs.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025, lrf=0.15, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_soil_det_hq, nbs=64, nms=False, opset=None, optimize=False, o

In [ ]:
# Validate on val split (standard)
metrics = model.val(data=ABS_YAML, imgsz=IMGSZ, plots=True, save_json=True)
print("Validation metrics:", metrics)

# TTA (Test-Time Augmentation) for a tiny boost at inference time
tta_preds = model.predict(
    source=model.trainer.val_loader.dataset.im_files[:12] if hasattr(model, "trainer") else None,
    imgsz=IMGSZ,
    conf=0.25,
    iou=0.5,
    tta=True,
    save=True,
    project=PROJECT_DIR,
    name="yolo11_soil_infer_tta"
)
print("TTA predictions saved to:", os.path.join(PROJECT_DIR, "yolo11_soil_infer_tta"))


NameError: name 'model' is not defined

In [ ]:
from ultralytics import YOLO
import os, glob, yaml, json

PROJECT_DIR = "/content/drive/MyDrive/yolo11_soil_runs"
ABS_YAML = "/content/data_abs.yaml"

# --- Step 1: Load best weights ---
def find_best_weights(project_dir, run_name_hint="yolo11_soil_det_hq"):
    hint = os.path.join(project_dir, run_name_hint, "weights", "best.pt")
    if os.path.exists(hint):
        return hint
    hits = glob.glob(os.path.join(project_dir, "**", "weights", "best.pt"), recursive=True)
    return sorted(hits, key=lambda p: os.path.getmtime(p))[-1] if hits else None

best = find_best_weights(PROJECT_DIR)
assert best, f"Best weights not found in {PROJECT_DIR}"
print("✅ Loaded model from:", best)
model = YOLO(best)

# --- Step 2: Read test split path from ABS_YAML ---
with open(ABS_YAML, "r") as f:
    cfg = yaml.safe_load(f)

# fallback if test not present
split_key = "test" if "test" in cfg else "val"
print(f"📂 Evaluating on '{split_key}' split...")

# --- Step 3: Run validation/evaluation ---
metrics = model.val(
    data=ABS_YAML,
    split=split_key,
    imgsz=IMGSZ if 'IMGSZ' in globals() else 640,
    conf=0.25,
    iou=0.5,
    save_json=True,
    plots=True,       # saves confusion matrix, PR curve, F1 curve
    project=PROJECT_DIR,
    name="eval_metrics"
)

# --- Step 4: Print key metrics nicely ---
print("\n📊 Evaluation Summary:")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")

# --- Step 5: Per-class metrics (optional) ---
if hasattr(metrics.box, "maps"):
    print("\nPer-class mAP50:")
    names = cfg.get("names", [])
    for i, v in enumerate(metrics.box.maps):
        cname = names[i] if i < len(names) else f"class_{i}"
        print(f"  {cname:20s}: {v:.4f}")

print("\n📁 Plots & JSON results saved in:", metrics.save_dir)


✅ Loaded model from: /content/drive/MyDrive/yolo11_soil_runs/yolo11_soil_det_hq/weights/best.pt


FileNotFoundError: [Errno 2] No such file or directory: '/content/data_abs.yaml'

In [ ]:

# --- 1) Read ABS_YAML to locate test/val paths ---
with open(ABS_YAML, "r") as f:
    cfg = yaml.safe_load(f)

def build_image_list(split_path):
    if split_path is None:
        return []
    if os.path.isfile(split_path) and split_path.endswith(".txt"):
        with open(split_path) as f:
            return [l.strip() for l in f if l.strip()]
    # folder path: accept either "<split>/images" or "<split>" directly
    img_dir = split_path
    if os.path.isdir(os.path.join(split_path, "images")):
        img_dir = os.path.join(split_path, "images")
    imgs = []
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp","*.JPG","*.PNG"):
        imgs += glob.glob(os.path.join(img_dir, ext))
    return imgs

test_imgs = build_image_list(cfg.get("test"))
if not test_imgs:
    print("⚠️ No images found in test split. Falling back to val.")
    test_imgs = build_image_list(cfg.get("val"))

assert len(test_imgs) > 0, "No images found in test/val splits. Check your paths."
random.shuffle(test_imgs)
sample_imgs = test_imgs[:8]
print(f"Using {len(sample_imgs)} sample images for evaluation.")
for p in sample_imgs:
    print(" -", p)


In [ ]:
# --- 2) Load the best weights (or reuse 'model' if it's already loaded) ---
def find_best_weights(project_dir, run_name_hint="yolo11_soil_det_hq"):
    # Try the tuned run name first, else any weights/best.pt we can find
    hint = os.path.join(project_dir, run_name_hint, "weights", "best.pt")
    if os.path.exists(hint):
        return hint
    # fallback: search any best.pt in the project dir
    hits = glob.glob(os.path.join(project_dir, "**", "weights", "best.pt"), recursive=True)
    return sorted(hits, key=lambda p: os.path.getmtime(p))[-1] if hits else None

try:
    _ = model  # if 'model' exists from training
    print("✅ Reusing the loaded model.")
except NameError:
    best = find_best_weights(PROJECT_DIR)
    assert best and os.path.exists(best), f"Could not find best.pt in {PROJECT_DIR}. Train first."
    print("✅ Loading model from:", best)
    model = YOLO(best)

In [ ]:
# --- 3) Run prediction (TTA optional for a small boost) ---
pred_out_name = "eval_test_samples"
pred = model.predict(
    source=sample_imgs,
    imgsz=IMGSZ if 'IMGSZ' in globals() else 640,
    conf=0.25,
    iou=0.5,
    save=True,
    project=PROJECT_DIR,
    name=pred_out_name,
    # tta=True,  # uncomment if you want Test-Time Augmentation
    verbose=False
)

# Determine save_dir robustly
save_dir = None
try:
    # ultralytics >= 8 returns a Results list; each has save_dir
    if pred and hasattr(pred[0], "save_dir"):
        save_dir = str(pred[0].save_dir)
except Exception:
    pass
if not save_dir:
    # fallback
    save_dir = os.path.join(PROJECT_DIR, pred_out_name)
print("🗂️ Predictions saved to:", save_dir)

# --- 4) Show detections inline in Colab ---
from IPython.display import display, Image
pngs = []
for ext in ("*.jpg","*.png","*.jpeg","*.bmp","*.webp"):
    pngs += glob.glob(os.path.join(save_dir, ext))

print(f"Found {len(pngs)} saved prediction images.")
for p in pngs:
    display(Image(filename=p))

In [ ]:
# Print a tidy summary of detections per sample image
from pathlib import Path

class_names = cfg.get("names", [])
def cname(i):
    return class_names[i] if 0 <= i < len(class_names) else str(i)

for i, res in enumerate(pred):
    im_path = sample_imgs[i] if i < len(sample_imgs) else "N/A"
    print("\n====", Path(im_path).name, "====")
    if res.boxes is None or len(res.boxes) == 0:
        print("No detections.")
        continue
    # xyxy, conf, cls
    for b in res.boxes:
        xyxy = b.xyxy[0].tolist()
        conf = float(b.conf[0])
        cls_id = int(b.cls[0])
        print(f"- {cname(cls_id)} @ {xyxy}  conf={conf:.3f}")


NameError: name 'cfg' is not defined